<a href="https://colab.research.google.com/github/BDR2939/NLP/blob/main/Assignment_3_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
Training a neural named entity recognition (NER) tagger 

In [ ]:
# ! pip install alive-progress

In [ ]:
import torch
import torch.nn as nn
import os 
import numpy as np
from random import shuffle
from sklearn.metrics import classification_report
from tqdm import tqdm
import pandas as pd
from google.colab import drive
from tqdm.notebook import tqdm_notebook
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# general folder path
drive_path = '/content/gdrive'
drive_saving_path = '/content/gdrive/My Drive'
drive.mount(drive_path)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive

/content/gdrive/My Drive


In [ ]:
! git clone https://github.com/BDR2939/NLP.git


Cloning into 'NLP'...
remote: Enumerating objects: 320, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 320 (delta 45), reused 61 (delta 24), pack-reused 237
Receiving objects: 100% (320/320), 1.41 MiB | 7.17 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
USER = 'DRIVE'  # 'OR' / 'RONY'
if USER == 'OR':
    main_folder =main_folder = r'C:\MSC\NLP2\HW3'
elif USER == 'DRIVE' :
    main_folder = folder_path = os.path.join(drive_saving_path, 'NLP/HW3/data')


In this assignment you are required to build a full training and testing pipeline for a neural sequentail tagger for named entities, using LSTM.

The dataset that you will be working on is called ReCoNLL 2003, which is a corrected version of the CoNLL 2003 dataset: https://www.clips.uantwerpen.be/conll2003/ner/

[Train data](https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing)

[Dev data](https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing)

[Test data](https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing)

As you can see, the annotated texts are labeled according to the IOB annotation scheme, for 3 entity types: Person, Organization, Location.

**Task 1:** Write a funtion for reading the data from a single file (of the ones that are provided above). The function recieves a filepath and then it encodes every sentence individually using a pair of lists, one list contains the words and one list contains the tags. Each list pair will be added to a general list (data), which will be returned back from the function.

## set path's

In [ ]:
train_path = os.path.join(main_folder, 'connl03_train.txt')
test_path = os.path.join(main_folder, 'connl03_test.txt')
dev_path = os.path.join(main_folder, 'connl03_dev.txt')

In [ ]:
def read_data(filepath):
    data = []
    with open(filepath) as file:
        words = []
        labels = []

        for index, line in enumerate(file, start=1):
            if line != '\n':
                word, label = line.split()
                words.append(word)
                labels.append(label)
            else:
                data.append((words, labels))
                words = []
                labels = []
    
    return data

train = read_data(train_path)
dev = read_data(test_path)
test = read_data(dev_path)

The following Vocab class can be served as a dictionary that maps words and tags into Ids. The UNK_TOKEN should be used for words that are not part of the training data.

In [ ]:
UNK_TOKEN = 0


class Vocab:
    def __init__(self):
        """
        tag2id/id2tag  - tags to each other from label to integer number
        n_words - count the # of word in sentence
        """
        self.word2id = {"__unk__": UNK_TOKEN}
        self.id2word = {UNK_TOKEN: "__unk__"}
        self.n_words = 1
        
        self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
        self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
    
    
    def index_words(self, words):
        """
        for given token list get token index in sentence
        """
        word_indexes = [self.index_word(w) for w in words]
        return word_indexes


    def index_tags(self, tags):
        """
        for given label list get label index
        """
        tag_indexes = [self.tag2id[t] for t in tags]
        return tag_indexes
    

    def index_word(self, w):
        """
     
        """
        if w not in self.word2id:
            self.word2id[w] = self.n_words
            self.id2word[self.n_words] = w
            self.n_words += 1
        
        return self.word2id[w]
    

**Task 2:** Write a function prepare_data that takes one of the [train, dev, test] and the Vocab instance, for converting each pair of (words,tags) to a pair of indexes. Each pair should be added to data_sequences, which will be returned back from the function.

In [ ]:
vocab = Vocab()

def prepare_data(data, vocab):
    data_sequences = []
    """
    this loop run on the data, for each sequence we generating tesor to
    contain the token of sequence
    """
    for i_words, i_tags in data:
        
        words_indexes_tensor = torch.tensor(vocab.index_words(i_words), dtype=torch.long)
        tags_indexes_tensor = torch.tensor(vocab.index_tags(i_tags), dtype=torch.long)
        # append data and label tensors
        data_sequences.append((words_indexes_tensor, tags_indexes_tensor))

    return data_sequences, vocab

train_sequences, vocab = prepare_data(train, vocab)
dev_sequences, vocab = prepare_data(dev, vocab)
test_sequences, vocab = prepare_data(test, vocab)

**Task 3:** Write NERNet, a PyTorch Module for labeling words with NER tags. 

*input_size:* the size of the vocabulary

*embedding_size:* the size of the embeddings

*hidden_size:* the LSTM hidden size

*output_size:* the number tags we are predicting for

*n_layers:* the number of layers we want to use in LSTM

*directions:* could 1 or 2, indicating unidirectional or bidirectional LSTM, respectively

The input for your forward function should be a single sentence tensor.

*note:* the embeddings in this section are learned embedding. That means that you don't need to use pretrained embedding like the one used in class. You will use them in part 5

In [ ]:
class NERNet(nn.Module):
    
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, directions):
        super(NERNet, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, bidirectional=(True if directions==2 else False))
        self.out = nn.Linear(hidden_size*directions, output_size)

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.directions = directions


    def forward(self, input_sentence):        
        # get sentence token numbers to understand output & input size
        dimension = len(input_sentence)
        
        # initial the hidden to None because none sentence inter
        hidden = None

        # 1. foward input sentence into the embeding
        embedded = self.embedding(input_sentence)

        # 2. foward embedding to LSTM
        lstm_output, _ = self.lstm(embedded.view(dimension, 1, -1), hidden) # The view function is meant to reshape the tensor https://stackoverflow.com/a/48650355/7786691

        # 3. foward to get predictions  - linear transformation to the incoming data
        output = self.out(lstm_output.view(dimension, -1)) 

        return output


## help function to train % evaluate 

In [ ]:
def get_model_results(model, test_sequences):
    """
    

    Parameters
    ----------
    model : Torch model  - 
        DESCRIPTION: LSTM model.
    test_sequences : list
        DESCRIPTION: input list of coupels [[word_tensor, lebel_tensor] , ...]
    
    the function get model results
    
    Returns
    -------
    all_test_words_pred : list
    all_test_words_true : list
    binary_test_words_pred : list
    binary_test_words_true : list
    """
    # generate test tokens prediction
    all_test_words_pred = []
    all_test_words_true = []

    # generate test binnary prediction
    binary_test_words_pred = []
    binary_test_words_true = []
    for sentence, labels in test_sequences:
        sentence_tensor = torch.LongTensor(sentence).cuda()
        labels_tensor = torch.LongTensor(labels).cuda()

        _, pred_labels = model(sentence_tensor).T.max(0)

        all_test_words_pred += pred_labels.tolist()
        all_test_words_true += labels.tolist()

        binary_test_words_pred += [1 if i >= 1 else i for i in all_test_words_pred]
        binary_test_words_true += [1 if i >= 1 else i for i in all_test_words_true]
    return all_test_words_pred, all_test_words_true, binary_test_words_pred, binary_test_words_true

**Task 4:** write a training loop, which takes a model (instance of NERNet) and number of epochs to train on. The loss is always CrossEntropyLoss and the optimizer is always Adam.

In [ ]:
def train_loop(model, n_epochs, train_sequences):
    #
    all_target_names = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]
    binary_target_names = ["O", "OTHERS"]
    
    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer (ADAM is a fancy version of SGD)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
  
    # shuffle data before training phase
    shuffle(train_sequences)
    STEP = 400 
    curr_f1_accuracy_result = 0
    best_f1_accuracy_result = 0
    best_df = pd.DataFrame()
    for e in range(1, n_epochs + 1):
        # print('start ephoc #' + str(e),  flush = True)
        """
        tqdm - add progress bar
        """
        desc = ('Ephoc #' + str(e))
        for sequence_idx in tqdm_notebook(range(train_sequences.__len__()), desc = desc):
            
            
            # get sentence tokens, and labels 
            sentence, labels = train_sequences[sequence_idx]
            
            # check if there is empty sentence
            if labels.__len__() == 0:
                continue
            
            # insert sentence tokens into tensor
            sentence_tensor = torch.LongTensor(sentence).cuda()
            
            # insert sentence labels into tensor
            labels_tensor = torch.LongTensor(labels).cuda()
            
            # Sets the gradients of all optimized to zero.
            model.zero_grad()
            
            # foward sentence to model
            scores = model(sentence_tensor)
            
            # Computes the gradient of current tensor
            criterion(scores, labels_tensor).backward()
            
            # once the gradients are computed use them to optimize model
            optimizer.step()

        
        # print('finshed ephoc #' + str(e) + ', ephoch results:' , flush = True)
        all_train_words_pred, all_train_words_true, \
        binary_train_words_pred, binary_train_words_true = get_model_results(model, train_sequences)
        train_Results_df = pd.DataFrame(classification_report(all_train_words_true, all_train_words_pred, target_names=all_target_names, output_dict = True))
        curr_f1_accuracy_result = train_Results_df.iloc[2]['accuracy']

        
        if curr_f1_accuracy_result > best_f1_accuracy_result:
          improve_string = 'f1-accuracy-score improve from ' + str(best_f1_accuracy_result) + ' to ' + str(curr_f1_accuracy_result) 
          best_f1_accuracy_result = curr_f1_accuracy_result
          best_df = train_Results_df
        else:
          improve_string = 'f1-accuracy-score did not improve from ' + str(best_f1_accuracy_result)  
        print(improve_string, flush = True)
       
    best_string = 'best-f1-accuracy-score is '+ str(best_f1_accuracy_result)  
    print(best_string, flush = True)
    return best_df


**Task 5:** write an evaluation loop on a trained model, using the dev and test datasets. This function print the true positive rate (TPR), also known as Recall and the opposite to false positive rate (FPR), also known as precision, of each label seperately (7 labels in total), and for all the 6 labels (except O) together. The caption argument for the function should be served for printing, so that when you print include it as a prefix.

In [ ]:
def evaluate(model, caption, test_sequences, dev_sequences):
    # from Piazza: https://piazza.com/class/klxc3m1tzqz2o8?cid=59

    all_target_names = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]
    binary_target_names = ["O", "OTHERS"]
    
    # self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
    # self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
    
    print(f"****************    Results for {caption}    ****************")



    # generate dev tokens prediction 
    all_dev_words_pred = []
    all_dev_words_true = []
    
    # generate dev binnary prediction 
    binary_dev_words_pred = []
    binary_dev_words_true = []

    # get test results
    all_test_words_pred, all_test_words_true, \
        binary_test_words_pred, binary_test_words_true = get_model_results(model, test_sequences)

    # get dev results
    all_dev_words_pred, all_dev_words_true, \
        binary_dev_words_pred, binary_dev_words_true = get_model_results(model, dev_sequences)

    print("Test Results:")
    Test_Results_dict = pd.DataFrame(classification_report(all_test_words_true, all_test_words_pred, target_names=all_target_names,  output_dict = True))
    display(Test_Results_dict.T)
    print("Dev Results:")
    Dev_Results_dict = pd.DataFrame(classification_report(all_dev_words_true, all_dev_words_pred, target_names=all_target_names, output_dict = True))
    display(Dev_Results_dict.T)

    print("Binary Test Results:")
    Binary_Test_Results = pd.DataFrame(classification_report(binary_test_words_true, binary_test_words_pred, target_names=binary_target_names, output_dict = True))
    display(Binary_Test_Results.T)

    print("Binary Dev Results:")
    Binary_Dev_Results  = pd.DataFrame(classification_report(binary_dev_words_true, binary_dev_words_pred, target_names=binary_target_names, output_dict = True))
    display(Binary_Dev_Results.T)

    return 

**Task 6:** Train and evaluate a few models, all with embedding_size=300, and with the following hyper parameters (you may use that as captions for the models as well):

Model 1: (hidden_size: 500, n_layers: 1, directions: 1)

Model 2: (hidden_size: 500, n_layers: 2, directions: 1)

Model 3: (hidden_size: 500, n_layers: 3, directions: 1)

Model 4: (hidden_size: 500, n_layers: 1, directions: 2)

Model 5: (hidden_size: 500, n_layers: 2, directions: 2)

Model 6: (hidden_size: 500, n_layers: 3, directions: 2)

Model 4: (hidden_size: 800, n_layers: 1, directions: 2)

Model 5: (hidden_size: 800, n_layers: 2, directions: 2)

Model 6: (hidden_size: 800, n_layers: 3, directions: 2)

In [ ]:
EMBEDDING_SIZE = 300
EPOCHS = 10
HIDDEN_SIZE  = 500 
INPUT_SIZE = len(vocab.word2id) # 8955
OUTPUT_SIZE = len(vocab.tag2id) # 7

n_layers_array = np.arange(1,4)
directions_array = np.arange(1,3)

# n_layers_array = np.arange(1,2)
# directions_array = np.arange(1,2)
model_list  = []
train_res_list = [] 
for i_n_layers in n_layers_array:
    for i_directions in directions_array:
        print('----------------------------------------------------------')
        print('Train model using:\n' + \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True)
        
        model = NERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, int(i_n_layers), int(i_directions)).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)
        model_list.append(model)
        train_res_list.append(train_res)

DIRECTION = 2
HIDDEN_SIZE = 800
for i_n_layers in n_layers_array:
        print('----------------------------------------------------------')

        print('Train model using:\n'+ \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True )
        model = NERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, i_n_layers, DIRECTION).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)
        model_list.append(model)

        

----------------------------------------------------------
Train model using:
  1)hidden_size = 500
  2)n_layers = 1
  3)directions = 1


Ephoc #1:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0 to 0.8380098452883263


Ephoc #2:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.8380098452883263 to 0.889381153305204


...

f1-accuracy-score improve from 0.9946554149085794 to 0.995182841068917


Ephoc #9:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score did not improve from 0.995182841068917


Ephoc #10:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.995182841068917 to 0.9954992967651195
best-f1-accuracy-score is 0.9954992967651195


## evaluate 


In [ ]:
for i, model in enumerate(model_list):
    model_name = "model_"+str(i)
    evaluate(model, model_name, test_sequences, dev_sequences)

****************    Results for model_0    ****************
Test Results:


,precision,recall,f1-score,support
O,0.915244,0.969638,0.941656,3096.000000
B-PER,0.688776,0.675000,0.681818,200.000000
I-PER,0.849558,0.611465,0.711111,157.000000
B-LOC,0.802548,0.688525,0.741176,183.000000
I-LOC,0.833333,0.434783,0.571429,23.000000
B-ORG,0.659420,0.541667,0.594771,168.000000
I-ORG,0.808511,0.327586,0.466258,116.000000
accuracy,0.887142,0.887142,0.887142,0.887142
macro avg,0.793913,0.606952,0.672603,3943.000000
weighted avg,0.881393,0.887142,0.879067,3943.000000


Dev Results:


,precision,recall,f1-score,support
O,0.922475,0.969392,0.945352,6567.000000
B-PER,0.712082,0.638249,0.673147,434.000000
I-PER,0.869565,0.675676,0.760456,296.000000
B-LOC,0.797342,0.699708,0.745342,343.000000
I-LOC,0.935484,0.547170,0.690476,53.000000
B-ORG,0.663230,0.551429,0.602184,350.000000
I-ORG,0.620000,0.310000,0.413333,200.000000
accuracy,0.893728,0.893728,0.893728,0.893728
macro avg,0.788597,0.627375,0.690041,8243.000000
weighted avg,0.886028,0.893728,0.886940,8243.000000


Binary Test Results:


...

,precision,recall,f1-score,support
O,0.954566,0.956601,0.955583,6567.000000
B-PER,0.714286,0.771889,0.741971,434.000000
I-PER,0.736842,0.804054,0.768982,296.000000
B-LOC,0.749304,0.784257,0.766382,343.000000
I-LOC,0.750000,0.622642,0.680412,53.000000
B-ORG,0.682927,0.640000,0.660767,350.000000
I-ORG,0.647482,0.450000,0.530973,200.000000
accuracy,0.906345,0.906345,0.906345,0.906345
macro avg,0.747915,0.718492,0.729296,8243.000000
weighted avg,0.905256,0.906345,0.905173,8243.000000


Binary Test Results:


,precision,recall,f1-score,support
O,0.947214,0.966102,0.956565,380877.000000
OTHERS,0.878258,0.819564,0.847896,113647.000000
accuracy,0.932426,0.932426,0.932426,0.932426
macro avg,0.912736,0.892833,0.902231,494524.000000
weighted avg,0.931367,0.932426,0.931591,494524.000000


Binary Dev Results:


,precision,recall,f1-score,support
O,0.953832,0.956756,0.955292,1.616269e+06
OTHERS,0.833199,0.823463,0.828303,4.239800e+05
accuracy,0.929057,0.929057,0.929057,9.290567e-01
macro avg,0.893516,0.890110,0.891797,2.040249e+06
weighted avg,0.928764,0.929057,0.928903,2.040249e+06


**Task 6:** Download the GloVe embeddings from https://nlp.stanford.edu/projects/glove/ (use the 300-dim vectors from glove.6B.zip). Then intialize the nn.Embedding module in your NERNet with these embeddings, so that you can start your training with pre-trained vectors. Repeat Task 6 and print the results for each model.

Note: make sure that vectors are aligned with the IDs in your Vocab, in other words, make sure that for example the word with ID 0 is the first vector in the GloVe matrix of vectors that you initialize nn.Embedding with. For a dicussion on how to do that, check it this link:
https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222

## get glove data set

In [ ]:

GLOVE_PATH = 'glove.6B.300d.txt'

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2022-07-05 13:45:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-07-05 13:45:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-07-05 13:45:46--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

# get embding weights

In [ ]:
def get_glove_pre_trained_embeddings_weights(input_size, embedding_size, word2id = vocab.word2id):
    # generate zeros initilized embeding mask 
    weights = np.zeros((input_size, embedding_size))

    # parse embeding
    with open(GLOVE_PATH) as glove:
        for line in glove.readlines():
            split = line.split()
            word = split[0]
            word_id = word2id.get(word)

        if word_id:
            weights[word_id] = split[1:]
    tensor = torch.from_numpy(weights).float()
    return tensor

## define Glove net

In [ ]:
class GloveNERNet(nn.Module):
    
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, directions):
        super(GloveNERNet, self).__init__()
        
        # add first layer - the embding layer
        self.embedding = nn.Embedding(input_size, embedding_size)

        # get trained embding weights from the data we loaded
        pre_trained_weights = get_glove_pre_trained_embeddings_weights(input_size, embedding_size)
        
        # load embding weights 
        self.embedding.weight = nn.Parameter(pre_trained_weights)

        # add LSTM layer
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, bidirectional=(True if directions==2 else False))
        
        # Add FC layer
        self.out = nn.Linear(hidden_size*directions, output_size)
    
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.directions = directions

    def forward(self, input_sentence):        
        # get sentence token numbers to understand output & input size
        dimension = len(input_sentence)
        
        # initial the hidden to None because none sentence inter
        hidden = None

        # 1. foward input sentence into the embeding
        embedded = self.embedding(input_sentence)

        # 2. foward embedding to LSTM
        lstm_output, _ = self.lstm(embedded.view(dimension, 1, -1), hidden) # The view function is meant to reshape the tensor https://stackoverflow.com/a/48650355/7786691

        # 3. foward to get predictions  - linear transformation to the incoming data
        output = self.out(lstm_output.view(dimension, -1)) 

        return output

## train model

In [ ]:
EMBEDDING_SIZE = 300
EPOCHS = 10
HIDDEN_SIZE  = 500 
INPUT_SIZE = len(vocab.word2id) # 8955
OUTPUT_SIZE = len(vocab.tag2id) # 7

n_layers_array = np.arange(1,4)
directions_array = np.arange(1,3)

# n_layers_array = np.arange(1,2)
# directions_array = np.arange(1,2)
model_list  = []
train_res_list = [] 
for i_n_layers in n_layers_array:
    for i_directions in directions_array:
        print('----------------------------------------------------------')
        print('Train model using:\n' + \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True)
        model = GloveNERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, 1, 1).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)   
        model_list.append(model)
        train_res_list.append(train_res)

DIRECTION = 2
HIDDEN_SIZE = 800
for i_n_layers in n_layers_array:
        print('----------------------------------------------------------')

        print('Train model using:\n'+ \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True )
        model = GloveNERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, 1, 1).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)   

       
        model_list.append(model)

        

----------------------------------------------------------
Train model using:
  1)hidden_size = 500
  2)n_layers = 1
  3)directions = 1


Ephoc #1:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0 to 0.8487341772151898


Ephoc #2:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.8487341772151898 to 0.9179676511954993


...

f1-accuracy-score improve from 0.985196905766526 to 0.9896976090014065


Ephoc #9:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.9896976090014065 to 0.9911040787623067


Ephoc #10:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score did not improve from 0.9911040787623067
best-f1-accuracy-score is 0.9911040787623067


## evaluate 


In [ ]:
for i, model in enumerate(model_list):
    model_name = "model_"+str(i)
    evaluate(model, model_name, test_sequences, dev_sequences)

****************    Results for model_0    ****************
Test Results:


,precision,recall,f1-score,support
O,0.975953,0.917636,0.945896,3096.000000
B-PER,0.909091,0.600000,0.722892,200.000000
I-PER,0.989474,0.598726,0.746032,157.000000
B-LOC,0.740741,0.765027,0.752688,183.000000
I-LOC,0.410256,0.695652,0.516129,23.000000
B-ORG,0.378613,0.779762,0.509728,168.000000
I-ORG,0.350649,0.698276,0.466859,116.000000
accuracy,0.868121,0.868121,0.868121,0.868121
macro avg,0.679254,0.722154,0.665746,3943.000000
weighted avg,0.915037,0.868121,0.882476,3943.000000


Dev Results:


,precision,recall,f1-score,support
O,0.984904,0.904066,0.942755,6567.000000
B-PER,0.946996,0.617512,0.747559,434.000000
I-PER,0.907317,0.628378,0.742515,296.000000
B-LOC,0.750000,0.778426,0.763948,343.000000
I-LOC,0.467532,0.679245,0.553846,53.000000
B-ORG,0.374834,0.808571,0.512217,350.000000
I-ORG,0.300557,0.810000,0.438430,200.000000
accuracy,0.866068,0.866068,0.866068,0.866068
macro avg,0.676020,0.746600,0.671610,8243.000000
weighted avg,0.924513,0.866068,0.884829,8243.000000


Binary Test Results:


...

,precision,recall,f1-score,support
O,0.992422,0.857469,0.920023,6567.000000
B-PER,0.979839,0.559908,0.712610,434.000000
I-PER,0.906736,0.591216,0.715746,296.000000
B-LOC,0.567347,0.810496,0.667467,343.000000
I-LOC,0.448276,0.735849,0.557143,53.000000
B-ORG,0.323496,0.814286,0.463038,350.000000
I-ORG,0.255224,0.855000,0.393103,200.000000
accuracy,0.827611,0.827611,0.827611,0.827611
macro avg,0.639048,0.746318,0.632733,8243.000000
weighted avg,0.921206,0.827611,0.856736,8243.000000


Binary Test Results:


,precision,recall,f1-score,support
O,0.989044,0.873397,0.927630,380877.00000
OTHERS,0.695161,0.967575,0.809053,113647.00000
accuracy,0.895040,0.895040,0.895040,0.89504
macro avg,0.842103,0.920486,0.868341,494524.00000
weighted avg,0.921506,0.895040,0.900380,494524.00000


Binary Dev Results:


,precision,recall,f1-score,support
O,0.992756,0.858601,0.920818,1.616269e+06
OTHERS,0.644238,0.976117,0.776190,4.239800e+05
accuracy,0.883022,0.883022,0.883022,8.830216e-01
macro avg,0.818497,0.917359,0.848504,2.040249e+06
weighted avg,0.920331,0.883022,0.890763,2.040249e+06


**Good luck!**